# Valorant API

<a href="https://dash.valorant-api.com/">site: Valorant API</a>

<p style="color: #aaa; font-size: 0.9rem; padding-left: 2rem; padding-right: 2rem;">
We provide an extensive API containing data of most in-game items, assets and more!
All data comes straight from the Valorant game files and will get updated automatically once another patch gets released.
If you have any questions and/or suggestions, you can find a way to contact us here.
</p>

<h3 style="color: #ccc">Import dependencies</h3>

In [1]:
import os
import sys
import urllib as ur
import json
import yaml


# set path sys default root 
sys.path.append(os.path.join('../..'))

# import dependencies local
from services.pymongo_service import PymongoService
from domain.valorant.model.agents_model import AgentsModel

<p style="color: #aaa; font-size: 0.9rem; padding-left: 2rem; padding-right: 2rem;">
first of all, let's doing test of connection with database mongodb, it's necessary to persist data, in this analysis we're going to use non_handler database name, because the information that we're going to fetch is not to our business. Just technical proposal.
Let's import our <strong>PymongoService</strong> that is responsible to management connection
</p>

In [71]:
# test connection
assert PymongoService().get_mongo_client().admin.command('ping')

<p style="color: #aaa; font-size: 0.9rem; padding-left: 2rem; padding-right: 2rem;">
Let's define some variables <strong>GLOBAL</strong> because we're goint to use them, in all our analysis
</p>

In [2]:
# storage path root and some constants variables
AGENTS = 'agents'
VALORANT = 'valorant'

ROOT_PATH = os.path.join('../..')
STORAGE_PATH = f'{ROOT_PATH}/storage'
VALORANT_PATH = f'{STORAGE_PATH}/{VALORANT}'
VALORANT_FILE_PATH = f'{VALORANT_PATH}/files'
VALORANT_IMAGE_PATH = f'{VALORANT_PATH}/images'


# create directory if not exits
for dir in [VALORANT_PATH, VALORANT_FILE_PATH, VALORANT_IMAGE_PATH]:
    if not os.path.exists(dir):
        os.mkdir(dir)

<p style="color: #aaa; font-size: 0.9rem; padding-left: 2rem; padding-right: 2rem;">
Let's started! - first of all we have to fetch some data from Valorant API - so we need to make connection with your application. this information about valorant is in a file <strong>application.yml</strong> that have resources.
how we do read an yaml file, need install lib yaml - <code>pip install PyYAML</code>
    <ul>
        <a href="https://stackoverflow.com/questions/1773805/how-can-i-parse-a-yaml-file-in-python"> 
            <li style="color: #ccc; font-size: 0.9rem;">https://stackoverflow.com/questions/1773805/how-can-i-parse-a-yaml-file-in-python</li>
        </a>
        <a href="https://pypi.org/project/PyYAML/"> 
            <li style="color: #ccc; font-size: 0.9rem;">https://pypi.org/project/PyYAML/</li>
        </a>
    <ul>
</p>

In [3]:
# read file yaml
with open(f'{ROOT_PATH}/application.yml') as file:
    root = yaml.safe_load(file)
    valorant = root.get('domain').get(VALORANT)
    mongo = root.get('infra').get('mongo')
    server = root.get('server')

    file.close()

In [74]:
display(valorant, mongo)

{'url': 'https://valorant-api.com/v1',
 'relative-paths': ['agents',
  'buddies',
  'bundles',
  'ceremonies',
  'competitivetiers',
  'contenttiers',
  'contracts',
  'currencies',
  'events',
  'gamemodes',
  'gear',
  'levelborders',
  'maps',
  'playercards',
  'playertitles',
  'seasons',
  'sprays',
  'themes',
  'weapons']}

{'driver': 'mongodb+srv://',
 'database': {'cluster': 'cluster0.otugy2g.mongodb.net/',
  'user': 'herdeiros_aurora',
  'pass': 'herdeiros_aurora_001',
  'names': ['ml_service_non_handler_db', 'dev_ml_service_db']}}

In [75]:
try:
    response = ur.request.urlopen(f'{valorant.get("url")}/{AGENTS}')
except Exception as e:
    print(str(e))

In [76]:
byte_data = response.read()
encoding = response.info().get_content_charset('utf8')
result = byte_data.decode(encoding)

In [77]:
json.loads(result)

{'status': 200,
 'data': [{'uuid': 'e370fa57-4757-3604-3648-499e1f642d3f',
   'displayName': 'Gekko',
   'description': 'Gekko the Angeleno leads a tight-knit crew of calamitous creatures. His buddies bound forward, scattering enemies out of the way, with Gekko chasing them down to regroup and go again.',
   'developerName': 'Aggrobot',
   'characterTags': None,
   'displayIcon': 'https://media.valorant-api.com/agents/e370fa57-4757-3604-3648-499e1f642d3f/displayicon.png',
   'displayIconSmall': 'https://media.valorant-api.com/agents/e370fa57-4757-3604-3648-499e1f642d3f/displayiconsmall.png',
   'bustPortrait': 'https://media.valorant-api.com/agents/e370fa57-4757-3604-3648-499e1f642d3f/fullportrait.png',
   'fullPortrait': 'https://media.valorant-api.com/agents/e370fa57-4757-3604-3648-499e1f642d3f/fullportrait.png',
   'fullPortraitV2': 'https://media.valorant-api.com/agents/e370fa57-4757-3604-3648-499e1f642d3f/fullportrait.png',
   'killfeedPortrait': 'https://media.valorant-api.com/ag

In [4]:
mongo_instance = PymongoService.get_instance()
non_handler_db = mongo_instance.get_db(f'{mongo.get("database").get("names")[0]}')

<p style="color: #aaa; font-size: 0.9rem; padding-left: 2rem; padding-right: 2rem;">
    <ul>
        <a href="https://www.mongodb.com/pt-br/docs/drivers/pymongo/"> 
            <li style="color: #ccc; font-size: 0.9rem;">https://www.mongodb.com/pt-br/docs/drivers/pymongo/</li>
        </a>
    <ul>
</p>

In [13]:
# created_at = datetime.datetime.now(pytz.timezone(server.get('timezone'))).strftime('%Y-%m-%dT%H:%M:%S')
# hash_validator = hashlib.sha256(result.encode('utf-8')).hexdigest()
# agents = { 'agents': json.loads(result), 'created_at':  created_at, 'hash_validator': hash_validator}
# agents = AgentsModel(json.dumps(json.loads(result)))

# if not len(list(non_handler_db['valorant'].find({'hash_validator': agents.hash_validator}).limit(1))) > 0:
#     non_handler_db['valorant'].insert_one(vars(agents))

In [79]:
def getData(url: str) -> str:
    """
    description:
        this method is responsible to make http call on api valorant and get some data
    args:
        url (str): URL to make http call
    return:
        http response from valorant and convert to string
    """
    response = ur.request.urlopen(url)
    byte_data = response.read()
    encoding = response.info().get_content_charset('utf8')

    return byte_data.decode(encoding)

def build_model_valorant(data) -> AgentsModel:
    """
    description:
        this method is responsible to make parse string from data coming valorant
        and return new instance of AgentsModel
    args:
        data (any): result from valorant http response
    return:
         AgentsModel
    """
    return AgentsModel(json.dumps(json.loads(data)))


def insert_data(collection_name, data) -> None:
    """
    description:
        this method is responsible to make insert into database
    args:
        collection_name (str): name of MongoDb collection
        data (any): data which will be inserted
    see: 
        https://www.mongodb.com/pt-br/docs/manual/reference/method/db.collection.insert/
    """
    non_handler_db[collection_name].insert_one(vars(data))

<p style="color: #aaa; font-size: 0.9rem; padding-left: 2rem; padding-right: 2rem;">
Note: First of all, we have to search for Agents in Database if result were less than zero, then we have to create new register of Valorant's agents collection otherwise we have check if property <code>hash_validator</code> is equal to agents created earlier
</p>

In [80]:
r"""
see:
    https://stackoverflow.com/questions/72323776/find-only-specific-key-from-mongodb-using-pymongo
    https://stackoverflow.com/questions/6570371/when-to-use-and-when-to-use-is
"""
agents_list = list(non_handler_db[VALORANT].find({AGENTS: {'$exists': True}}))

if len(agents_list) <= 0:
    insert_data('valorant', 
                build_model_valorant(
                    getData(f'{valorant.get("url")}/{AGENTS}')))
else:
    for agent in agents_list:
        ag = AgentsModel(json.dumps(agent.get(AGENTS)))
        if ag.hash_validator != agent.get('hash_validator'):
            # make request to valorant to get new informations
            try:
                insert_data(VALORANT, 
                            build_model_valorant(
                                getData(f'{valorant.get("url")}/{AGENTS}')))
            except Exception as e:
                print(f"Erro: not was possible to fetch dada from {str(e)}")

<h3 style="color: #ccc">Fazendo pré-preparo para analise dos dados obtidos</h3>

In [38]:
import pandas as pd
import csv

In [40]:
collection = non_handler_db.get_collection(VALORANT)
result = collection.find({AGENTS: {'$exists': True}}).distinct(AGENTS)
list(result[0].get('data')[0].keys())

['uuid',
 'displayName',
 'description',
 'developerName',
 'characterTags',
 'displayIcon',
 'displayIconSmall',
 'bustPortrait',
 'fullPortrait',
 'fullPortraitV2',
 'killfeedPortrait',
 'background',
 'backgroundGradientColors',
 'assetPath',
 'isFullPortraitRightFacing',
 'isPlayableCharacter',
 'isAvailableForTest',
 'isBaseContent',
 'role',
 'recruitmentData',
 'abilities',
 'voiceLine']

In [32]:
columns = [
 'displayName',
 'description',
 'developerName',
 'characterTags',
 'displayIcon',
 'displayIconSmall',
 'bustPortrait',
 'fullPortrait',
 'fullPortraitV2',
 'killfeedPortrait',
 'background',
 'isFullPortraitRightFacing',
 'isPlayableCharacter',
 'isAvailableForTest',
 'isBaseContent',
 'voiceLine']

In [34]:
df = pd.DataFrame.from_records(result[0].get('data'), columns=columns)

In [39]:
df.to_csv(f'{VALORANT_FILE_PATH}/valorant_agents.0.0.1.csv', 
          sep=';', 
          index = False, 
          encoding = 'utf-8-sig', 
          quoting=csv.QUOTE_NONNUMERIC)

<h3 style="color: #ccc">Fase de Data Science - Analisando os dados e gerando insights</h3>